<a href="https://colab.research.google.com/github/ayush7garg/good-friday-gifts-hackathon/blob/master/good_friday_gifts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Importing Libraries
import pandas as pd
import numpy as np
import io
from google.colab import files

In [0]:
uploaded = files.upload()

Saving test.csv to test.csv


In [0]:
df = pd.read_csv('train.csv')

FileNotFoundError: ignored

In [0]:
df.head(15)

,gift_id,gift_type,gift_category,gift_cluster,instock_date,stock_update_date,lsg_1,lsg_2,lsg_3,lsg_4,lsg_5,lsg_6,uk_date1,uk_date2,is_discounted,volumes,price
0,GF_11156,61,534,3942,2014-02-21 05:07:06.000,2016-11-09 15:49:51.000,3377,5221,504,1912,10,554,2014-02-24 08:07:06.000,2014-02-24 07:07:06.000,0,NaN,175.54
1,GF_11157,61,534,3942,2014-02-21 06:07:06.000,2016-11-11 13:49:51.000,3377,5221,504,1912,10,554,2014-02-22 07:07:06.000,2014-02-24 06:07:06.000,1,NaN,95.80
2,GF_15689,584,262,0,2014-02-21 09:30:21.000,2016-03-24 14:46:18.000,5290,1579,3203,1912,9,1578,2016-01-26 00:04:45.000,2016-03-18 02:00:00.000,1,NaN,107.35
3,GF_11155,61,534,3942,2014-02-22 05:07:06.000,2016-11-10 16:49:51.000,3377,5221,504,1912,10,554,2016-11-07 13:49:51.000,2016-11-06 04:00:00.000,0,NaN,172.90
4,GF_11158,61,534,3942,2014-02-22 07:07:06.000,2016-11-10 13:49:51.000,3377,5221,504,1912,9,554,2016-11-07 15:49:51.000,2016-11-06 01:00:00.000,1,NaN,77.72
5,GF_15686,584,262,0,2014-02-23 07:30:21.000,2016-03-23 15:46:18.000,5290,1579,3203,1912,9,1578,2014-02-25 07:30:21.000,2014-02-24 08:30:21.000,1,NaN,142.84
6,GF_15690,584,262,0,2014-02-23 10:30:21.000,2016-03-23 18:46:18.000,5290,1579,3203,1912,9,1578,2016-01-25 02:04:45.000,2016-03-16 02:00:00.000,1,NaN,78.71
7,GF_15685,584,262,0,2014-02-24 10:30:21.000,2016-03-24 17:46:18.000,5290,1579,3203,1912,9,1578,2016-01-22 03:04:45.000,2016-03-17 03:00:00.000,0,NaN,166.81
8,GF_11159,61,534,3942,2014-02-25 07:07:06.000,2016-11-10 14:49:51.000,3377,5221,504,1912,10,554,2016-11-07 16:49:51.000,2015-04-08 12:48:27.025,0,NaN,72.72
9,GF_15688,584,262,0,2014-02-25 08:30:21.000,2016-03-24 16:46:18.000,5290,1579,3203,1912,9,1578,2016-01-23 03:04:45.000,2016-03-11 04:00:00.000,1,NaN,107.12


In [0]:
df['volumes'] = df['volumes'].fillna(0)

In [0]:
df.head(5)

,gift_id,gift_type,gift_category,gift_cluster,instock_date,stock_update_date,lsg_1,lsg_2,lsg_3,lsg_4,lsg_5,lsg_6,uk_date1,uk_date2,is_discounted,volumes,price
0,GF_11156,61,534,3942,2014-02-21 05:07:06.000,2016-11-09 15:49:51.000,3377,5221,504,1912,10,554,2014-02-24 08:07:06.000,2014-02-24 07:07:06.000,0,0.0,175.54
1,GF_11157,61,534,3942,2014-02-21 06:07:06.000,2016-11-11 13:49:51.000,3377,5221,504,1912,10,554,2014-02-22 07:07:06.000,2014-02-24 06:07:06.000,1,0.0,95.80
2,GF_15689,584,262,0,2014-02-21 09:30:21.000,2016-03-24 14:46:18.000,5290,1579,3203,1912,9,1578,2016-01-26 00:04:45.000,2016-03-18 02:00:00.000,1,0.0,107.35
3,GF_11155,61,534,3942,2014-02-22 05:07:06.000,2016-11-10 16:49:51.000,3377,5221,504,1912,10,554,2016-11-07 13:49:51.000,2016-11-06 04:00:00.000,0,0.0,172.90
4,GF_11158,61,534,3942,2014-02-22 07:07:06.000,2016-11-10 13:49:51.000,3377,5221,504,1912,9,554,2016-11-07 15:49:51.000,2016-11-06 01:00:00.000,1,0.0,77.72


In [0]:
len(df.gift_type.unique())

1237

In [0]:
x_train = df.drop("price",axis=1)
y_train = df["price"]